<a href="https://colab.research.google.com/github/ozturkcemal/SupplyChainAnalytics/blob/main/02_Linear_Programming/SupplierSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyomo

In [ ]:
!apt-get install -y -qq coinor-cbc

Include Pyomo

In [24]:
import pyomo.environ as pyo

Define sets, indices

In [25]:
#sets
Source={'Saudi','Venezuela'}
Products={'Gasoline','Jet Fuel','Lubricant'}

#input parameters
capacity={'Saudi':9,'Venezuela':6}
cost={'Saudi':20,'Venezuela':15}
demand={'Gasoline':2,'Jet Fuel':1.5,'Lubricant':0.5}
transform={
     ('Saudi', 'Gasoline'): 0.3,
    ('Saudi', 'Jet Fuel'): 0.4,
    ('Saudi', 'Lubricant'): 0.2,
    ('Venezuela', 'Gasoline'): 0.4,
    ('Venezuela', 'Jet Fuel'): 0.2,
    ('Venezuela', 'Lubricant'): 0.3
}

Create the model instance

In [26]:
model = pyo.ConcreteModel()

Define variables

In [27]:
model.x = pyo.Var( Source, within=pyo.NonNegativeReals)
model.x.pprint()

x : Size=2, Index={Saudi, Venezuela}
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
        Saudi :     0 :  None :  None : False :  True : NonNegativeReals
    Venezuela :     0 :  None :  None : False :  True : NonNegativeReals


Define objective function

In [28]:
model.obj = pyo.Objective(
    expr = sum( cost[i]*model.x[i] for i in Source ),
    sense = pyo.minimize )

model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 20*x[Saudi] + 15*x[Venezuela]


Define capacity constraints

In [29]:
model.constraint_capacity = pyo.ConstraintList()
for i in Source:
  model.constraint_capacity.add(model.x[i] <= capacity[i])
model.constraint_capacity.pprint()

constraint_capacity : Size=2, Index={1, 2}, Active=True
    Key : Lower : Body         : Upper : Active
      1 :  -Inf :     x[Saudi] :   9.0 :   True
      2 :  -Inf : x[Venezuela] :   6.0 :   True


Define demand constraints

In [30]:
model.constraint_demand = pyo.ConstraintList()
for j in Products:
  model.constraint_demand.add(sum(transform[i,j]*model.x[i] for i in Source) >= demand[j])
model.constraint_demand.pprint()

constraint_demand : Size=3, Index={1, 2, 3}, Active=True
    Key : Lower : Body                            : Upper : Active
      1 :   0.5 : 0.2*x[Saudi] + 0.3*x[Venezuela] :  +Inf :   True
      2 :   2.0 : 0.3*x[Saudi] + 0.4*x[Venezuela] :  +Inf :   True
      3 :   1.5 : 0.4*x[Saudi] + 0.2*x[Venezuela] :  +Inf :   True


Define solver environment

In [38]:
opt = pyo.SolverFactory('cbc')
model.pprint()

1 Var Declarations
    x : Size=2, Index={Saudi, Venezuela}
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
            Saudi :     0 :   2.0 :  None : False : False : NonNegativeReals
        Venezuela :     0 :   3.5 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 20*x[Saudi] + 15*x[Venezuela]

2 Constraint Declarations
    constraint_capacity : Size=2, Index={1, 2}, Active=True
        Key : Lower : Body         : Upper : Active
          1 :  -Inf :     x[Saudi] :   9.0 :   True
          2 :  -Inf : x[Venezuela] :   6.0 :   True
    constraint_demand : Size=3, Index={1, 2, 3}, Active=True
        Key : Lower : Body                            : Upper : Active
          1 :   0.5 : 0.2*x[Saudi] + 0.3*x[Venezuela] :  +Inf :   True
          2 :   2.0 : 0.3*x[Saudi] + 0.4*x[Venezuela] :  +Inf :   True
          3 :   1.5 : 0

Solve the model

In [39]:
opt_solution = opt.solve(model)

Print solution

In [40]:
# Print the values of the decision variables
print("\nSupply amounts:")
for i in Source:
    print(f"{i}: {model.x[i].value}")

print(f"Total cost: {model.obj()}")


Supply amounts:
Saudi: 2.0
Venezuela: 3.5
Total cost: 92.5
